[![AWS Data Wrangler](_static/logo.png "AWS Data Wrangler")](https://github.com/awslabs/aws-data-wrangler)

# 27 - Amazon Timestream - Example 2

## Reading test data

In [1]:
import awswrangler as wr
import pandas as pd
from datetime import datetime

df = pd.read_csv(
    "https://raw.githubusercontent.com/awslabs/amazon-timestream-tools/master/sample_apps/data/sample.csv",
    names=[
        "ignore0",
        "region",
        "ignore1",
        "az",
        "ignore2",
        "hostname",
        "measure_kind",
        "measure",
        "ignore3",
        "ignore4",
        "ignore5",
    ],
    usecols=["region", "az", "hostname", "measure_kind", "measure"],
)
df["time"] = datetime.now()
df.reset_index(inplace=True, drop=False)

df

,index,region,az,hostname,measure_kind,measure,time
0,0,us-east-1,us-east-1a,host-fj2hx,cpu_utilization,21.394363,2020-12-06 21:21:17.846716
1,1,us-east-1,us-east-1a,host-fj2hx,memory_utilization,68.563420,2020-12-06 21:21:17.846716
2,2,us-east-1,us-east-1a,host-6kMPE,cpu_utilization,17.144579,2020-12-06 21:21:17.846716
3,3,us-east-1,us-east-1a,host-6kMPE,memory_utilization,73.507870,2020-12-06 21:21:17.846716
4,4,us-east-1,us-east-1a,host-sxj7X,cpu_utilization,26.584865,2020-12-06 21:21:17.846716
...,...,...,...,...,...,...,...
125995,125995,eu-north-1,eu-north-1c,host-De8RB,memory_utilization,68.063468,2020-12-06 21:21:17.846716
125996,125996,eu-north-1,eu-north-1c,host-2z8tn,memory_utilization,72.203680,2020-12-06 21:21:17.846716
125997,125997,eu-north-1,eu-north-1c,host-2z8tn,cpu_utilization,29.212219,2020-12-06 21:21:17.846716
125998,125998,eu-north-1,eu-north-1c,host-9FczW,memory_utilization,71.746134,2020-12-06 21:21:17.846716


## Write CPU_UTILIZATION records

In [2]:
df_cpu = df[df.measure_kind == "cpu_utilization"].copy()
df_cpu.rename(columns={"measure": "cpu_utilization"}, inplace=True)

rejected_records = wr.timestream.write(
    df=df_cpu,
    database="sampleDB",
    table="sampleTable",
    time_col="time",
    measure_col="cpu_utilization",
    dimensions_cols=["index", "region", "az", "hostname"],
)

assert len(rejected_records) == 0

## Write MEMORY_UTILIZATION records

In [3]:
df_memory = df[df.measure_kind == "memory_utilization"].copy()
df_memory.rename(columns={"measure": "memory_utilization"}, inplace=True)

rejected_records = wr.timestream.write(
    df=df_memory,
    database="sampleDB",
    table="sampleTable",
    time_col="time",
    measure_col="memory_utilization",
    dimensions_cols=["index", "region", "az", "hostname"],
)

assert len(rejected_records) == 0

## Querying CPU_UTILIZATION

In [4]:
wr.timestream.query("""
    SELECT 
        hostname, region, az, measure_name, measure_value::double, time
    FROM "sampleDB"."sampleTable"
    WHERE measure_name = 'cpu_utilization'
    ORDER BY time DESC
    LIMIT 10
""")

,hostname,region,az,measure_name,measure_value::double,time
0,host-dCRm0,us-west-1,us-west-1a,cpu_utilization,85.793809,2020-12-06 21:21:17.847
1,host-JI1Tc,us-east-2,us-east-2a,cpu_utilization,61.584796,2020-12-06 21:21:17.847
2,host-YL3W0,eu-central-1,eu-central-1a,cpu_utilization,63.921383,2020-12-06 21:21:17.847
3,host-Axf0g,us-west-2,us-west-2a,cpu_utilization,36.962473,2020-12-06 21:21:17.847
4,host-Bwb3j,us-east-1,us-east-1a,cpu_utilization,45.082867,2020-12-06 21:21:17.847
5,host-39Hem,ap-east-1,ap-east-1c,cpu_utilization,86.057493,2020-12-06 21:21:17.847
6,host-93VEW,us-east-2,us-east-2c,cpu_utilization,20.691806,2020-12-06 21:21:17.847
7,host-lPyRj,eu-central-1,eu-central-1b,cpu_utilization,97.424692,2020-12-06 21:21:17.847
8,host-c45yb,us-west-2,us-west-2a,cpu_utilization,87.205414,2020-12-06 21:21:17.847
9,host-bSljV,us-east-2,us-east-2a,cpu_utilization,27.825704,2020-12-06 21:21:17.847


## Querying MEMORY_UTILIZATION

In [5]:
wr.timestream.query("""
    SELECT 
        hostname, region, az, measure_name, measure_value::double, time
    FROM "sampleDB"."sampleTable"
    WHERE measure_name = 'memory_utilization'
    ORDER BY time DESC
    LIMIT 10
""")

,hostname,region,az,measure_name,measure_value::double,time
0,host-AyzHS,eu-central-1,eu-central-1b,memory_utilization,83.209870,2020-12-06 21:21:17.847
1,host-EREJk,ap-east-1,ap-east-1a,memory_utilization,36.529079,2020-12-06 21:21:17.847
2,host-YL3W0,eu-central-1,eu-central-1a,memory_utilization,32.010117,2020-12-06 21:21:17.847
3,host-BM6B6,eu-north-1,eu-north-1a,memory_utilization,80.981302,2020-12-06 21:21:17.847
4,host-HE2Gm,eu-north-1,eu-north-1a,memory_utilization,22.555852,2020-12-06 21:21:17.847
5,host-RdQRf,us-east-1,us-east-1c,memory_utilization,47.356774,2020-12-06 21:21:17.847
6,host-24Gju,us-east-1,us-east-1b,memory_utilization,56.868000,2020-12-06 21:21:17.847
7,host-WgAuL,us-east-2,us-east-2a,memory_utilization,57.562315,2020-12-06 21:21:17.847
8,host-t4XZj,eu-north-1,eu-north-1b,memory_utilization,66.307538,2020-12-06 21:21:17.847
9,host-wWlQ3,eu-north-1,eu-north-1a,memory_utilization,18.807051,2020-12-06 21:21:17.847
